## Processing Parallel Corpora

The entire data processing in this project is built on the **datatrove** library from **huggingface**.
The main storage for one data sample in datatrove is the Document class:

```python
class Document:
    text: str
    id: str
    media: list[Media] = field(default_factory=list)
    metadata: dict[str, Any] = field(default_factory=dict)
```

When you start working, you specify which column of your dataset you want to work with, after which this column is renamed to "text", and all other data goes into Document.metadata.

**Data Transformation Example:**

Original Dataset:
```python
{
    "ru": "Привет, мир!",
    "chv": "Сывлăх, тĕнче!",
    "random_shit": 42
}
```

When selecting "ru" as the key column:

```python
doc.text = "Привет, мир!"
doc.metadata = {
    'chv': "Сывлăх, тĕнче!",
    'random_shit': 42
}
```

Visual representation:
```
┌───────────────────────┐                 ┌───────────────────────────────┐
│      Original Data    │                 │          Document             │
│                       │                 │                               │
│  "ru": "Привет, мир!" │─────┐           │  text: "Привет, мир!"         │
│                       │     │           │                               │
│  "chv": "Сывлăх, ..." │     │           │  metadata: {                  │
│                       │     └──────────>│    "chv": "Сывлăх, тĕнче!",   │
│  "random_shit": 42    │                 │    "random_shit": 42          │
│                       │                 │  }                            │
└───────────────────────┘                 └───────────────────────────────┘

```

If you want to work with multiple columns simultaneously, datatrove in most cases cannot provide you with such an opportunity - in some basic classes, for example, it is hardcoded that work is done exclusively on Document.text.

Eeve implements, based on some blocks of datatrove, such blocks that allow you to directly specify which column from your dataset you would like to transform.

In [1]:
import re
import random
import pandas as pd

from datatrove.pipeline.readers.csv import CsvReader
from datatrove.pipeline.writers.jsonl import JsonlWriter
from datatrove.executor.local import LocalPipelineExecutor

from eeve.utils.datatrove import (
    _reader_adapter_with_column_info,
    _writer_adapter_with_column_restore
)

from eeve.data.formatters.callable import CallableFormatter
from eeve.data.formatters.quote import QuoteReplacer

In [2]:
def print_random_rows(df_list, k):
    m = min(len(df) for df in df_list)
    idx = random.sample(range(m), k)
    n = min(df.shape[1] for df in df_list)
    a = df_list[0]
    b = df_list[1] if len(df_list) == 2 else None
    for c in range(n):
        print(list(a.iloc[idx, c]))
        if b is not None:
            print(list(b.iloc[idx, c]))
        print()

In [3]:
inputs = pd.read_csv("./files/parallel_corpus_process.csv")
inputs.head(7)

,text_1,text_2
0,"That professor built ""the historical"" document.","The cat mysteriously ""built the"" ancient artif..."
1,The government explained the natural ...,Our team discovered the interesting th...
2,The beautiful the company walked the a...,"This computer powerfully designed the ""natural..."
3,A student powerfully investigated the ...,The interesting this computer walked t...
4,A student quickly discovered an innova...,Our team investigated a challenging puzzle...
5,"Several birds interestingly ""created the natur...",The powerful that professor created a ch...
6,"The cat studied the ""ancient artifact quickly.""",My neighbor created the historical docu...


In [4]:
print_random_rows([inputs], k=2)

['That professor built "the historical" document.', '"The government" developed the mathematical equation.']

['The cat mysteriously "built the" ancient artifact.', 'A student    walked    the  amazing    the   complex    problem.']



To get a dataset after transformations not in the format that datatrove provides, but in the original format (preserving all columns and their names), you can use adapters. The _reader_adapter_with_column_info adapter is used when reading data and saves information about the original columns and their names in metadata, while the _writer_adapter_with_column_restore adapter takes this information into account when writing data.

NB: to get data in the same format at the output as you submitted at the input, you need to use **both reader and writer adapters** in the pipeline. An example will be shown below.

In [5]:
example_reader = CsvReader("./files/parallel_corpus_process.csv", text_key="text_1")
gen = example_reader.run()

doc = next(iter(gen))
print(doc) # as we can see, all values from the dataset, except text_key='text_1', have been transferred to doc.metadata

2025-08-19 22:34:46.301 | INFO     | datatrove.pipeline.readers.base:read_files_shard:201 - Reading input file , 1/1


Document(text='That professor built "the historical" document.', id='/0', media=[], metadata={'text_2': 'The cat mysteriously "built the" ancient artifact.', 'file_path': 'd:/vs_projects/eeve/examples/files/parallel_corpus_process.csv'})


In the demonstration dataset above, we can see several text formatting issues:
- chevron quotes need to be properly placed;
- multiple spaces need to be replaced;
- there are indentations before the first or after the last character in a sentence.

Let's implement functions that remove these deficiencies, and use them as examples to transform the columns in our corpus.

In [6]:
def process_string(s: str) -> str:
    s = re.sub(r'\s+', ' ', s)
    return s.strip()

In [7]:
executor = LocalPipelineExecutor(
    pipeline=[
        CsvReader(
            "./files/parallel_corpus_process.csv",
            text_key="text_1",
            adapter=_reader_adapter_with_column_info
        ),
        CallableFormatter(func=process_string, list_path=["text", "metadata['text_2']"]),
        QuoteReplacer(list_path=["text", "metadata['text_2']"]),
        JsonlWriter(
            output_folder="./files",
            output_filename='result_${rank}',
            adapter=_writer_adapter_with_column_restore
        )
    ],
    tasks=1,
    workers=1,
    skip_completed=False
)

In [8]:
executor.run()

2025-08-19 22:34:51.035 | INFO     | datatrove.utils.logging:add_task_logger:58 - Launching pipeline for rank=0
2025-08-19 22:34:51.037 | INFO     | datatrove.utils.logging:log_pipeline:90 - 
--- 🛠️ PIPELINE 🛠
📖 - READER: 🔢 Csv
✂️ - FORMAT: 🔧 Custom
✂️ - FORMAT: 💬 Quotes
💽 - WRITER: 🐿 Jsonl
--- Logging error in Loguru Handler #3 ---
Record was: {'elapsed': datetime.timedelta(seconds=7, microseconds=938448), 'exception': None, 'extra': {}, 'file': (name='logging.py', path='c:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\datatrove\\utils\\logging.py'), 'function': 'log_pipeline', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 90, 'message': '\n--- 🛠️ PIPELINE 🛠\n📖 - READER: 🔢 Csv\n✂️ - FORMAT: 🔧 Custom\n✂️ - FORMAT: 💬 Quotes\n💽 - WRITER: 🐿 Jsonl', 'module': 'logging', 'name': 'datatrove.utils.logging', 'process': (id=1416, name='MainProcess'), 'thread': (id=21032, name='MainThread'), 'time': datetime(2025, 8, 19, 22, 34, 51, 37312, tzinfo=datetime.tim



📉📉📉 Stats 📉📉📉

Total Runtime: 0 seconds

📖 - READER: 🔢 Csv
    Runtime: (6.50%) 0 seconds [0.02 milliseconds±0.01 milliseconds/doc]
    Stats: {input_files: 1, doc_len: 2730 [min=31, max=76, 54.60±9/doc], documents: 50 [50.00/input_file]}
✂️ - FORMAT: 🔧 Custom
    Runtime: (24.34%) 0 seconds [0.06 milliseconds±0.03 milliseconds/doc]
    Stats: {total: 50}
✂️ - FORMAT: 💬 Quotes
    Runtime: (21.65%) 0 seconds [0.06 milliseconds±0.02 milliseconds/doc]
    Stats: {total: 50}
💽 - WRITER: 🐿 Jsonl
    Runtime: (47.51%) 0 seconds [0.12 milliseconds±0.49 milliseconds/doc]
    Stats: {result_XXXXX.gz: 50, total: 50, doc_len: 2537 [min=30, max=62, 50.74±7/doc]}

In [9]:
result = pd.read_json(path_or_buf='./files/result_00000.gz', lines=True)

In [10]:
print_random_rows([inputs, result], k=2)

['Several birds interestingly "created the natural" phenomenon.', 'Several   birds    analyzed an   innovative solution.']
['Several birds interestingly «created the natural» phenomenon.', 'Several birds analyzed an innovative solution.']

['The powerful that  professor  created    a  challenging puzzle. ', ' The colorful the company analyzed the complex problem.']
['The powerful that professor created a challenging puzzle.', 'The colorful the company analyzed the complex problem.']

